1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
3. [Добавление новых столбцов](#new)
4. [Проверка не попадают ли какие-то пользователи во все группы](#doble)  
5. [Изучение логов](#log)
6. [Временной интервал в A/A/B-тесте](#date)
7. [Количество потерянных событий и пользователей](#lost)
8. [Воронка событий](#vor)
9. [Изучение результатов эксперимента](#rez)
10. [Вывод по результатам теста](#rez_test)
11. [Общий вывод](#done)

## Открытие данных <a id="start"></a> 

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import math as mth
import plotly.express as px
from matplotlib import pyplot as plt
from scipy import stats as st
from plotly import graph_objects as go

In [ ]:
# загрузка данных
try:
    logs_exp = pd.read_csv('/datasets/logs_exp.csv', sep = '\t')  
except:
    logs_exp = pd.read_csv('https://code.s3.yandex.net/datasets/logs_exp.csv', sep = '\t')

In [ ]:
# функция для предобработки данных
def data_preprocessing(data):
    # приведение наименования столбцов датафрейма к правильному виду
    data.columns = [x.lower().replace(' ', '_') for x in data.columns]
    print('Вывод первых 10 строчек датафрейма на экран')  
    display(data.head(10))
    print(' ')   
    print('Вывод основной информации о датафрейме с помощью метода info()') 
    display(data.info())
    print(' ') 
    print('Вывод количества пропущенных значений для каждого столбца датафрейма')
    display(data.isna().sum())
    print(' ') 
    print('Подсчёт количества явных дубликатов')
    display(data.duplicated().sum())

In [ ]:
data_preprocessing(logs_exp)

**Вывод**  
Наименования столбцов необходимо привести к читаемому виду. Необходимо столбец 'expid' привести к типу 'object'. Пропущенных значений в датафрейме нет. Явных дубликатов — 413. Необходимо их удалить. Также для удобства лучше переименовать численное обозначение групп на буквенное. 246 и 247 — контрольные группы (заменим на А и АA соответственно), а 248 — экспериментальная (на В).

## Предобработка данных <a id="preprocessing"></a> 

In [ ]:
# приведение наименования столбцов датафрейма к необходимому виду
logs_exp = logs_exp.rename(
    columns={
        'eventname' : 'event', 
        'deviceidhash' : 'user_id', 
        'eventtimestamp' : 'event_time',
        'expid' : 'group'
    }
)

In [ ]:
# замена значений контрольных и эксперементальной групп 
# 246 и 247 — контрольные группы (А и АA соответственно), а 248 — экспериментальная (В)
# создание функции для замены значений контрольных и эксперементальной групп
def group_name(logs_exp):
    if logs_exp['group'] == 246:
        return 'A'
    elif logs_exp['group'] == 247:
        return 'AA'
    else:
        return 'B'

logs_exp['group'] = logs_exp.apply(group_name, axis=1)
logs_exp.head(10)

In [ ]:
# пропусков нет

# изменение типа данных в столбце датафрейма, используя обработку ошибок при приобразовании
try:
    logs_exp['group'] = logs_exp['group'].astype('object')
except:
    print('group - ошибка')  

In [ ]:
logs_exp.head(10)

In [ ]:
# удаление дубликатов
logs_exp = logs_exp.drop_duplicates()
logs_exp.duplicated().sum()

### Добавление новых столбцов <a id="new"></a> 

In [ ]:
# добавление столбца даты и времени
logs_exp['date_time'] = pd.to_datetime(logs_exp['event_time'], unit='s')

In [ ]:
# добавление столбца дат
logs_exp['date'] = pd.to_datetime(logs_exp['date_time']).dt.date

In [ ]:
# проверка данных после всех манипуляций
logs_exp.head(10)

In [ ]:
# проверка типов данных после всех манипуляций
logs_exp.info()

In [ ]:
# изменение типа данных в столбце датафрейма, используя обработку ошибок при приобразовании
try:
    logs_exp['date'] = logs_exp['date'].astype('datetime64')
except:
    print('date - ошибка')  

In [ ]:
# проверка типов данных после всех манипуляций
logs_exp.info()

**Вывод**  
Была проведена предобработка данных:  
* изменены типы данных group и date;  
* добавлены столбцы date_time и date;  
* удалены дубликаты;  
* наименования столбцов приведены к необходимому виду;  
* назначены буквенные значения для групп.

### Проверка не попадают ли какие-то пользователи во все группы <a id="doble"></a> 

In [ ]:
# проверка не попадают ли какие-то пользователи во все группы

In [ ]:
print('Виды групп:', logs_exp['group'].unique())

In [ ]:
# количество пользователей в каждой группе
group_a = logs_exp.query('group == "A"')['user_id'].count()
print('Количество пользователей в группе А:', group_a)

group_aa = logs_exp.query('group == "AA"')['user_id'].count()
print('Количество пользователей в группе АA:', group_aa)

group_b = logs_exp.query('group == "B"')['user_id'].count()
print('Количество пользователей в группе B:', group_b)

In [ ]:
# количество уникальных пользователей в каждой группе
group_a_unique = logs_exp.query('group == "A"')['user_id'].nunique()
print('Количество уникальных пользователей в группе А:', group_a_unique)

group_aa_unique = logs_exp.query('group == "AA"')['user_id'].nunique()
print('Количество уникальных пользователей в группе АA:', group_aa_unique)

group_b_unique = logs_exp.query('group == "B"')['user_id'].nunique()
print('Количество уникальных пользователей в группе B:', group_b_unique)

group_unique = logs_exp['user_id'].nunique()
print('Количество уникальных пользователей всего:', group_unique)

group_sum = group_a_unique + group_aa_unique + group_b_unique
print('Количество уникальных пользователей всего суммирование:', group_sum)

In [ ]:
# пользователи, попавшие во все группы
# формируем группы
group_a = logs_exp.query('group == "A"')
group_aa = logs_exp.query('group == "AA"')
group_b = logs_exp.query('group == "B"')

In [ ]:
# отображаем id пользователей, попавших во все группы
group_ab = np.intersect1d(group_a['user_id'], group_b['user_id'])
print('id пользователей, попавших в группы A, B:', group_ab)
group_aaa = np.intersect1d(group_a['user_id'], group_aa['user_id'])
print('id пользователей, попавших в группы A, AA:', group_aaa)
group_aab = np.intersect1d(group_aa['user_id'], group_b['user_id'])
print('id пользователей, попавших в группы A, AA:', group_aab)

**Вывод**   
В тесте участвуют группы А, AA, В. Количество уникальных пользователей, а также проверка присутствия id пользователей одновременно в нескольких группах говорят о том, что пользователи не попали в несколько групп одновременно.

### Изучение логов <a id="log"></a> 

In [ ]:
logs_exp['event'].value_counts()

In [ ]:
event_all = logs_exp['event'].count()
print('Всего событий в логе', event_all)

event_unique = logs_exp['event'].nunique()
print('Количество уникальных событий в логе', event_unique)

event_name = logs_exp['event'].unique()
print('Наименования уникальных событий в логе', event_name)

In [ ]:
group_unique = logs_exp['user_id'].nunique()
print('Количество уникальных пользователей всего:', group_unique)

In [ ]:
# медианное значение, чтобы исключить возможность искажения результатов из-за выбросов
mean = logs_exp.groupby('user_id')['event'].count().median()
print('Среднее количество событий на пользователя', mean)

### Временной интервал в A/A/B-тесте <a id="date"></a> 

In [ ]:
# временной интервал в A/A/B-тесте
# дата начала теста
first = logs_exp['date'].min()
print('Дата начала теста:', first)

# дата окончания теста
last = logs_exp['date'].max()
print('Дата окончания теста:', last)

# период теста
period = last - first
print('Период теста в днях:', period.days)

**Вывод**   
Минимальная дата теста 25 июля 2019 года, а максимальная дата — 7 августа 2019 года. Таким образом, мы располагаем данными за перод около двух недель. 

In [ ]:
logs_exp.hist('date', grid=True)
plt.title('Распределение событий по дате и времени')
plt.xlabel('Дата')
plt.ylabel('Событие')
plt.xticks(rotation=30)
plt.show()

**Вывод**   
На основании графика можно сделать вывод, что мы располагаем данными с 29 июля 2019 года, однако до 1 августа 2019 года тенденция неустойчивая и рост событий приходится на промежуток 31 июля 2019 - 1 августа 2019. Возможно до 1 августа 2019 года данные не фиксировались по какой-то ошибке. Таким образом полные данные начинаются с 1 августа 2019 года. 

In [ ]:
# исключаем данные до 01 августа 2019 года
logs_exp_new = logs_exp.query('date >= "2019-08-01"').reset_index(drop=True)
logs_exp_new.head(10)

In [ ]:
# временной интервал в A/A/B-тесте после исключения данных до 01 августа 2019 года
# дата начала теста
first = logs_exp_new['date'].min()
print('Дата начала теста:', first)

# дата окончания теста
last = logs_exp_new['date'].max()
print('Дата окончания теста:', last)

# период теста
period = last - first
print('Период теста в днях:', period.days)

In [ ]:
# проверка распределения после исключения данных до 01 августа 2019 года
logs_exp_new.hist('date', grid=True)
plt.title('Распределение событий по дате и времени')
plt.xlabel('Дата')
plt.ylabel('Событие')
plt.xticks(rotation=30)
plt.show()

**Вывод**   
Таким образом, после исключения данных до 1 августа 2019 года, можно увидеть примерно равно распределение событий. Минимальная дата теста 1 августа 2019 года, а максимальная дата — 7 августа 2019 года. Мы располагаем данными за перод 6 дней.

### Количество потерянных событий и пользователей <a id="lost"></a> 

In [ ]:
# количество потерянных событий

event_all = logs_exp['event'].count()
print('Всего событий до удаления', event_all)

event_unique = logs_exp['event'].nunique()
print('Количество уникальных событий до удаления', event_unique)

event_all_after = logs_exp_new['event'].count()
print('Всего событий после удаления', event_all_after)

event_unique_after = logs_exp_new['event'].nunique()
print('Количество уникальных событий после удаления', event_unique_after)

result_event = event_all - event_all_after
print('Количество потерянных событий:', result_event)

result_event_pr = round(((event_all - event_all_after)/event_all * 100), 2)
print(f'Количество потерянных событий в процентах: {result_event_pr}%')

In [ ]:
# количество потерянных пользователей

group_unique = logs_exp['user_id'].nunique()
print('Количество пользователей до удаления:', group_unique)

group_unique_after = logs_exp_new['user_id'].nunique()
print('Количество пользователей после удаления:', group_unique_after)

result_group = group_unique - group_unique_after
print('Количество потерянных пользователей:', result_group)

result_group_pr = round(((group_unique - group_unique_after)/group_unique * 100), 2)
print(f'Количество потерянных пользователей в процентах: {result_group_pr}%')

**Вывод**   
Можно сделать вывод, была потеряна небольшая доля пользователей и событий. После удаления было потеряно 1.16% событий, а также 0.23% полользователей.

In [ ]:
print('Виды групп:', logs_exp_new['group'].unique())

**Вывод**   
Пользователи из всех трех групп остались в очищенных данных.

**Общий вывод**   
Минимальная дата теста до удаления данных была 25 июля 2019 года, а максимальная дата — 7 августа 2019 года. Таким образом, мы располагали данными за перод около двух недель. Однако, после удаления нерелевантных данных до 1 августа 2019 года, минимальная дата теста сместилась на 1 августа 2019 года. Удаление данных до 1 августа 2019 года не привело к значительным потерям данных: 0.23% пользователей и 1.16% событий. Также данное удаление не повлекло за собой потерю какой-либо группы тестирования.

## Воронка событий <a id="vor"></a> 

In [ ]:
# частота событий в логах и количества уникальных пользователей на каждом этапе
count_new = logs_exp_new.groupby('event').agg({'event':'count', 'user_id':'nunique'}).sort_values(by='user_id', ascending=False)
count_new.columns = ['events_count', 'users_count']
count_new = count_new.reset_index()
count_new

**Вывод**   
Наиболее часто встречающееся действие — «MainScreenAppear», а наименее популярное — «Tutorial».

In [ ]:
# отображение количества уникальных пользователей на каждом этапе
count_new

**Вывод**  
В предыдущем пункте собрали табличку для отображения частоты событий в логах и количества уникальных пользователей на каждом этапе. На основании её можно сделать вывод о количестве уникальных пользователей на каждом этапе.  
* MainScreenAppear - Появление главного экрана.
* OffersScreenAppear - Появление предложения на экране.
* CartScreenAppear - Появление экрана корзины.  
* PaymentScreenSuccessful - Экран успешно проведенного платежа.
* Tutorial - Руководство пользователя.  
Наиболее часто встречающееся действие — «MainScreenAppear» т.е появление главного экрана, а наименее популярное — «Tutorial» т.е. руководство пользователя. Наиболее часто и редко встречающиеся действия вполне логичны по количеству совершенных переходов.

In [ ]:
# доля пользователей, которые хоть раз совершали событие
count_new ['share'] = round((count_new['users_count'] / group_unique_after * 100), 2)
count_new

**Вывод**  
Наибольшая доля пользователей 98.47% совершили действие «MainScreenAppear - Появление главного экрана». Наименьшую долю пользователей совершили такое действие, как «Tutorial - Руководство пользователя».  
Около 1.5% пользователей миновали главный экран, возможно, по причине того, что эти данные об этих пользователях содержались в данных, которые были удалены.

Действия происходят в таком порядке:  
* MainScreenAppear (Появление главного экрана): сначала пользователь открыл приложение;  
* OffersScreenAppear (Появление предложения на экране): открыл необходимый товар;  
* CartScreenAppear (Появление экрана корзины): добавил товар в корзину и открылся экран корзины;  
* PaymentScreenSuccessful (Экран успешно проведенного платежа):  провел оплату;  
* Tutorial (Руководство пользователя):  открыл инструкцию (возможно инструкция к получению товара).  
  
Последовательность действий с одной стороны выглядит логичной, а с другой, даже на личном опыте понятно, что перед тем, как провести оплату, пользователи должны прочесть инструкцию, а не после.  
Таким образом, «Tutorial» не входит в воронку событий и не является обязательным действием для совершения покупки.  
  
Следовательно, действия можно представить в такой последовательности:  
Действия происходят в таком порядке:  
* MainScreenAppear (Появление главного экрана): сначала пользователь открыл приложение;  
* OffersScreenAppear (Появление предложения на экране): открыл необходимый товар;  
* CartScreenAppear (Появление экрана корзины): добавил товар в корзину и открылся экран корзины;  
* PaymentScreenSuccessful (Экран успешно проведенного платежа):  провел оплату.

In [ ]:
# исключение Tutorial (Руководство пользователя) при рассчете вопронки событий
logs_exp_new = logs_exp_new.query('event != "Tutorial"')

In [ ]:
# преобразование в лист количества пользователей на каждом действии 
list_pers = list(logs_exp_new.groupby('event')['user_id'].nunique().sort_values(ascending=False))
list_pers

In [ ]:
# вывод доли пользователей
print(f'Доля пользователей перешедших с главного экрана в каталог товаров составляет\
      {round((list_pers[1] / list_pers[0] * 100), 2)}%')
print(f'Доля пользователей перешедших из каталога товаров в корзину товаров составляет\
      {round((list_pers[2] / list_pers[1] * 100), 2)}%')
print(f'Доля пользователей перешедших из корзины товаров на страницу оплаты составляет\
      {round((list_pers[3] / list_pers[2] * 100), 2)}%')

In [ ]:
# исключение из частоты событий в логах и количества уникальных пользователей этапа 'Tutorial' для построения воронки событий
count_new = count_new.query('event != "Tutorial"')

In [ ]:
# воронка событий
fig = go.Figure(go.Funnel(
    x = count_new['users_count'], 
    y = count_new['event'], 
    textinfo = "value+percent initial+percent previous",
    marker = {"color": ["deepskyblue", "lightsalmon", "tan", "teal", "silver"],
    "line": {"width": [4, 2, 2, 3, 1, 1], "color": ["wheat", "wheat", "blue", "wheat", "wheat"]}}
)
               )
fig.update_layout(title={'text': "Воронка событий"})
fig.show();

**Вывод**  
Наибольшая доля пользователей переходит из корзины товаров на страницу оплаты — 94.78%, а наименьшая доля переходящих пользователей составляют переходящие с главного экрана в каталог товаров (61.91%).

In [ ]:
# вывод доли потерянных пользователей
print(f'Доля потерянных пользователей не перешедших с главного экрана в каталог товаров составляет\
      {round(100-(list_pers[1] / list_pers[0] * 100), 2)}%')
print(f'Доля потерянных пользователей не перешедших из каталога товаров в корзину товаров составляет\
      {round(100-(list_pers[2] / list_pers[1] * 100), 2)}%')
print(f'Доля потерянных пользователей не перешедших из корзины товаров на страницу оплаты составляет\
      {round(100-(list_pers[3] / list_pers[2] * 100), 2)}%')

**Вывод**  
Можно сделать вывод, что больше всего пользователей теряется на этапе перехода с главного экрана в каталог товаров — 38.09%.

In [ ]:
print(f'Доля пользователей перешедших с главного экрана на страницу оплаты составляет\
      {round((list_pers[3] / list_pers[0] * 100), 2)}%')

**Вывод**  
Можно сделать вывод, что меньше половины пользователей первый раз посетивших приложение перешли на страницу оплаты — 47.7%.

**Общий вывод**   
* Наиболее часто встречающееся событие — «MainScreenAppear» (доля пользователей 98.25% совершили действие) т.е появление главного экрана, а наименее популярное — «Tutorial» т.е. руководство пользователя.  
* В целом, можно проследить такую цепочку событий: сначала пользователь открывает приложение («MainScreenAppear»), затем страницу необходимого товара («OffersScreenAppear»). После этого пользователь добавляет товар в корзину («CartScreenAppear») и оплачивает («PaymentScreenSuccessful»). В самую последнюю очередь пользователь открывает руководство пользователя («Tutorial»).  
* По моему мнению, руководство пользователя («Tutorial»), в которое пользователи заходят по итогу всех действий, является нелогичным завершением действий пользователя, поскольку в данный раздел пользователи зашли бы, скорее, перед покупкой.  
* В связи с этим «Tutorial» был исключен из анализа последовательности действий пользователя.
* Наибольшая доля пользователей переходит из корзины товаров на страницу оплаты — 94.78%, а наименьшая доля переходящих пользователей (исключая «Tutorial») составляют пользователи переходящие с главного экрана в каталог товаров — 61.91%.  
* Следует упомянуть, что больше всего пользователей теряется на этапе перехода с главного экрана в каталог товаров, а, в целом, меньше половины пользователей первый раз посетивших приложение перешли на страницу оплаты — 47.7%.

## Изучение результатов эксперимента <a id="rez"></a> 

In [ ]:
# количество пользователей в каждой экспериментальной группе
users_group_test = logs_exp_new.groupby('group')['user_id'].nunique()
users_group_test['A+AA'] = users_group_test['A'] + users_group_test['AA']
users_group_test['all'] = logs_exp_new['user_id'].nunique()
users_group_test['all_sum'] = users_group_test['A+AA'] + users_group_test['B']
users_group_test

**Вывод**  
Таким образом, пользователей в группах:  
* 246 - 2483;  
* 247 - 2512;  
* 248 - 2535.

In [ ]:
difference = (1 - users_group_test['A'] / users_group_test['AA']) * 100
print(f'Разница группами A и AA составляет {round((difference), 2)}%')

**Вывод**  
Разница между выборками А и АА чуть больше 1%, соответственно, она незначительная.

In [ ]:
# отображение самого популярного события
event_for_groups = logs_exp_new.pivot_table(index='event', columns='group', values='user_id', aggfunc='nunique')\
                               .sort_values(by='A', ascending=False)\
                               .reset_index()

event_for_groups

**Вывод**    
Разница между группами А, АА, B незначительная, они, примерно, пропорциональны друг другу.

**Гипотезы:**   
* Нулевая гипотеза — значимых различий в долях пользователей нет;  
* Альтернативная — значимые различия в долях пользователей есть.  
  
Значение alpha = 0.05 (5%).  
Критерий проверки гитотез — равенство долей.  
Для проверки долей используется z-тест.

In [ ]:
# добавление данных в сводную таблицу для удобства рассчетов
event_for_groups['A+AA'] = event_for_groups['A'] + event_for_groups['AA']
event_for_groups['all_groups'] = event_for_groups['A+AA'] + event_for_groups['B']

event_for_groups['shares_A'] = (event_for_groups['A'] / group_a_unique * 100).round(2)
event_for_groups['shares_AA'] = (event_for_groups['AA'] / group_aa_unique * 100).round(2)
event_for_groups['shares_B'] = (event_for_groups['B'] / group_b_unique * 100).round(2)
event_for_groups['shares_A+AA'] = ((event_for_groups['A'] + event_for_groups['AA']) / \
                                  (group_a_unique + group_aa_unique) * 100).round(2)
event_for_groups

In [ ]:
fig = go.Figure()

fig.add_trace(go.Funnel(name = 'Группа A',
    y = event_for_groups["event"],
    x = event_for_groups["A"],textposition = "inside", 
    textinfo = "value+percent initial+percent previous"))

fig.add_trace(go.Funnel(name = 'Группа AA',
    y = event_for_groups["event"],
    x = event_for_groups["AA"],textposition = "inside", 
    textinfo = "value+percent initial+percent previous"))

fig.add_trace(go.Funnel(name = 'Группа B',
    y = event_for_groups["event"],
    x = event_for_groups["AA"],textposition = "inside", 
    textinfo = "value+percent initial+percent previous"))

fig.update_layout(title={'text': "Воронка событий с разбивкой по группам"})
fig.show();

**Вывод**    
На основании таблицы и воронки событий с разбивкой по группам, также можно сделать вывод, что наиболее популярное событие — MainScreenAppear. Группа A собершила самое популярное действие 2450 раз, AA — 2476 раз, а группа B — 2493 раза.

In [ ]:
# функция для статистической значимости

def z_test(group_1, group_2, alpha):
    #перебор экспериментальных групп по событиям и количество пользователей
    for i in event_for_groups.index:
        # пропорция успехов в 1 группе:
        p1 = event_for_groups[group_1][i] / users_group_test[group_1]
        # пропорция успехов во второй группе:
        p2 = event_for_groups[group_2][i] / users_group_test[group_2]
        # пропорция успехов в комбинированном датасете:
        p_combined = ((event_for_groups[group_1][i] + event_for_groups[group_2][i]) / 
                      (users_group_test[group_1] + users_group_test[group_2]))
        # разница пропорций в датасетах
        difference = p1 - p2
        # считаем статистику в ст.отклонениях стандартного нормального распределения
        z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * 
                                        (1/users_group_test[group_1] + 1/users_group_test[group_2]))
        # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
        distr = st.norm(0, 1) 
        p_value = (1 - distr.cdf(abs(z_value))) * 2
        print('{} p-значение: {}'.format(event_for_groups['event'][i], p_value))
        if (p_value < alpha):
            print("Отвергаем нулевую гипотезу: значимые различия в долях пользователей есть")
        else:
            print("Не отвергаем нулевую гипотезу: значимых различий в долях пользователей нет")

In [ ]:
z_test("A", "AA", 0.05)

In [ ]:
# проведем проверку со значением alpha = 0.01 (1%), чтобы убедиться в правильности эксперемента, 
# поскольку группы A и AA очень близки по значениям
z_test("A", "AA", 0.01)

**Вывод**    
Таким образом, можно сделать вывод, что тест и разбиение на группы было выполнено корректно. Статистически значимых различий в долях пользователей на каждом виде событий нет. Группы A и AA можно обе считать контрольными.

In [ ]:
z_test("A", "B", 0.05)

**Вывод**    
Значимых различий в долях пользователей между контрольной группой A (246) и экспериментальной группой B (248) нет.

In [ ]:
z_test("AA", "B", 0.05)

**Вывод**    
Значимых различий в долях пользователей между контрольной группой AA (247) и экспериментальной группой B (248) нет.

In [ ]:
z_test("A+AA", "B", 0.05)

**Вывод**    
Значимых различий в долях пользователей между объединённой контрольной группой A+AA (246+247) и экспериментальной группой B (248) нет.

### Вывод по результатам теста <a id="rez_test"></a> 

Было проведено 16 проверок:  
* 8 проверок между контрольными группами A (246), AA (247) и экспериментальной группой B (248);  
* 4 проверки между объединённой контрольной группой A+AA (246+247) и экспериментальной группой B (248);  
* 4 проверки между контрольными группами A (246) и AA (247).  
  
Для сравнения контрольных групп A и AA (246 и 247 соответственно) были использованы оба уровня статистической значимости (и 5% и 1%). Однако нулевую гипотезу так и не удалось отвергнуть.  
При проведении теста контрольных групп, объединенной контрольной группы и экспериментальной группы был использован уровень статистической значимости — 5%. Однако при проведении тестов, также не удалось отвергнуть нулевую гипотезу, следовательно, смена шрифта никак не повлияла на пользователей.  
Т.е. по результатам теста: новые шрифты не вызовут негативных изменений в поведении пользователей.

## Общий вывод <a id="done"></a> 

При проведении исследования были выполнены поставленные задачи:

* обработаны и подготовлены данные для проведения анализа.
* исследованы и проверены данные для исследования.
* изучена воронка событий.
* проанализированы результаты теста.  
* проанализированы самые прибыльные жанры игр.  
* принято решение о необходимости смены шрифтов во всём приложении по результатам теста.  
  
 
* Для удобства проведения исследвоания численные наименования групп были заменены на буквенные: контрольные группы A, AA, а эксперементальная — B.  
* В начале исследования был определен промежуток исследования в период около двух недель (минимальная дата теста 25 июля 2019 года, а максимальная дата — 7 августа 2019 года). Однако, на основании графиков, был сделан вывод, что полные данные начинаются с 1 августа 2019 года (поскольку до 1 августа 2019 года тенденция неустойчивая и рост событий приходится на промежуток 31 июля 2019 - 1 августа 2019). Такая ситуация могла сложиться из-за того, что события не фиксировались, возможно, была какая-то ошибка. Таким образом, минимальная дата теста 1 августа 2019 года, а максимальная дата — 7 августа 2019 года. Мы располагаем данными за перод 6 дней.  
* Наиболее часто встречающееся событие — «MainScreenAppear» (доля пользователей 98.25% совершили действие) т.е появление главного экрана, а наименее популярное — «Tutorial» т.е. руководство пользователя.  
* Цепочка действий пользователя: сначала пользователь открывает приложение («MainScreenAppear»), затем страницу необходимого товара («OffersScreenAppear»). После этого пользователь добавляет товар в корзину («CartScreenAppear») и оплачивает («PaymentScreenSuccessful»). В самую последнюю очередь пользователь открывает руководство пользователя («Tutorial»).   
* Руководство пользователя («Tutorial»), в которое пользователи заходят по итогу всех действий, является нелогичным завершением действий пользователя, поскольку в данный раздел пользователи зашли бы, скорее, перед покупкой. Данное событие было исклбчено из вронки событий. 
* Больше всего пользователей теряется на этапе перехода с главного экрана в каталог товаров. Если учитывать переход на страницу «Tutorial» (Руководство пользователя), то данная позиция была бы лидирующей по данному критерию — 76.26%.  
* Меньше половины пользователей первый раз посетивших приложение перешли на страницу оплаты — 47.7%.  
* Было проведено 16 проверок гипотез (включая проверку контрольных групп и объединенной контрольной группы с эксперементальной).  
* При проведении тестов, не удалось отвергнуть нулевую гипотезу, следовательно, новые шрифты не вызовут негативных изменений в поведении пользователей.